In [2]:
from pyspark.sql import functions as  f

StatementMeta(, be5e9e7f-136d-4995-844b-e830b20bf693, 6, Finished, Available)

In [3]:
class silver:
    def __init__(self):
        self.base_bronze_delta= 'abfss://cf927cbd-ab77-4aa6-9ebf-d7125aace954@onelake.dfs.fabric.microsoft.com/0f844c8f-7cd3-4525-93e2-e6923cd667f6/Files/bronzeDelta/'
        self.base_checkpoint='abfss://cf927cbd-ab77-4aa6-9ebf-d7125aace954@onelake.dfs.fabric.microsoft.com/7432d6ed-1dd7-41f0-8b94-3ce6ca527366/Files/checkpoint'
        self.base_siliver_delta= 'abfss://cf927cbd-ab77-4aa6-9ebf-d7125aace954@onelake.dfs.fabric.microsoft.com/7432d6ed-1dd7-41f0-8b94-3ce6ca527366/Files'


   

    def ExploratoryDA(self):
        # Your existing code to create and start the new stream
        # Read data from the temporary view vProductDetails
        product = spark.sql("select * from vProductDetails")
        
        # Read streaming data and calculate discount percentage
        sOrder = (spark.readStream.table("streaming_order")
                    .withColumn('discountPC', f.col('TotalAmount') / 100).drop('TotalAmount')
                )
        
        # Join product and streaming order data, calculate discount amount and actual cost
        joinDF = product.join(f.broadcast(sOrder), on='ProductID') \
                       .withColumn("discountAmt", f.round((f.col('StandardCost') * f.col('discountPC')) / 100, 2).cast("float")) \
                       .withColumn("ActualCost", f.col('StandardCost') - f.col('discountAmt')) \
                       .select(
                                'CustomerID',
                                'OrderID',
                                'ProductID',
                                'ProductNumber',
                                'OrderDate',
                                'standardCost',
                                'discountPC',
                                'DiscountAmt',
                                'ActualCost',
                                'Name',
                                'Description',
                                'PaymentMethod' )

        # Define write stream query for silver layer ingestion
        wQuery = joinDF.writeStream.queryName("silver-ingestion") \
                      .option("checkpointLocation", f"{self.base_checkpoint}/checkpoint/sl_order/") \
                      .outputMode("append") \
                      .start(f"{self.base_siliver_delta}/sl_order")
        
        

        # Assign the new stream query to self.stream_query
        return wQuery







StatementMeta(, be5e9e7f-136d-4995-844b-e830b20bf693, 7, Finished, Available)

In [4]:
sl=silver()
wQuery=sl.ExploratoryDA()

StatementMeta(, be5e9e7f-136d-4995-844b-e830b20bf693, 8, Finished, Available)


 create a view of **`vProductDetails**`
        -Get the current date price 

In [62]:
%%sql 
-- Create a temporary view for the joined data
-- Using CTEs to perform the required operations

-- Define the CTE for the joined data
create or replace view vProductDetails AS 
WITH joined_data AS (
    SELECT 
        prd.ProductID,
        prd.Name,
        prd.ProductNumber,
        prd.Description,
        product_cost.standardCost,
        product_cost.endDate,
        product_cost.StartDate
    FROM
        product prd
    JOIN 
        product_cost
    ON 
        prd.ProductID = product_cost.Product_ID
),

-- Define the CTE for the ranked data
ranked_data AS (
    SELECT
        ProductID,
        Name,
        ProductNumber,
        Description,
        standardCost,
        endDate,
        StartDate,
        RANK() OVER (PARTITION BY ProductID ORDER BY StartDate DESC) as rank
    FROM
        joined_data
)

select ProductID,
        Name,
        ProductNumber,
        Description,
        standardCost,
        endDate
       from ranked_data where rank=1








StatementMeta(, 297cd311-2bfd-4d2e-a384-2492f164fa71, 67, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [ ]:
prd=spark.sql("select * from vProductDetails")
sorder=spark.sql("SELECT * from streaming_order").withColumn('discountPC', f.col('TotalAmount')/100).drop('TotalAmount')


sorder.printSchema()

In [48]:
from pyspark.sql.window import Window
window_spec = Window.partitionBy("ProductID").orderBy(f.desc("StartDate"))


maskDf=(
    prd.join(
        product_cost,
        f.col('ProductID') == f.col('Product_ID')
    ))

rankDf=maskDf.withColumn('rank', f.rank().over(window_spec)).filter(f.col('rank')==1)

current_productDetails=rankDf.select('productID','Name','ProductNumber', 'standardCost', 'endDate','Description')

StatementMeta(, 297cd311-2bfd-4d2e-a384-2492f164fa71, 51, Finished, Available)

In [ ]:
display(prd.join(
    f.broadcast(sorder), on ='ProductID'
).withColumn(
    "discountAmt", f.round((f.col('StandardCost') * f.col('discountPC')) / 100, 2).cast("float")
)
.withColumn("ActualCost", f.col('StandardCost')-f.col('discountAmt')).printSchema())

In [9]:
%%sql
drop table silver.sl_order

StatementMeta(, 0731dec2-5f73-4da3-b262-2d6e46ff27af, 13, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [ ]:

%%sql
CREATE TABLE IF NOT EXISTS silver.sl_order
USING DELTA 
LOCATION 'abfss://cf927cbd-ab77-4aa6-9ebf-d7125aace954@onelake.dfs.fabric.microsoft.com/7432d6ed-1dd7-41f0-8b94-3ce6ca527366/Files/sl_order/';


In [14]:
%%sql

REFRESH TABLE silver.sl_order;


StatementMeta(, 0731dec2-5f73-4da3-b262-2d6e46ff27af, 18, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [16]:
%%sql
-- Optimize the silver.sl_order table by Z-Ordering based on the ProductID column
OPTIMIZE silver.sl_order ZORDER BY (CustomerID);


StatementMeta(, 0731dec2-5f73-4da3-b262-2d6e46ff27af, 20, Finished, Available)

<Spark SQL result set with 1 rows and 2 fields>

get the total number of orders in 30 mints with an window of 5 minutes 


In [ ]:
rQuery=spark.read.table("streaming_order")

In [27]:
rQuery.show()

StatementMeta(, be5e9e7f-136d-4995-844b-e830b20bf693, 31, Finished, Available)

+----------+--------------------+-------------------+-----------+----------------+---------+
|CustomerID|           OrderDate|            OrderID|TotalAmount|   PaymentMethod|ProductID|
+----------+--------------------+-------------------+-----------+----------------+---------+
|   1940692|2024-06-04 01:11:...|8694041981307296165|      864.0|Cash on Delivery|      839|
|   7740973|2024-06-04 01:11:...|4566235409339699499|      415.0|          PayPal|      332|
|   4837263|2024-06-04 01:11:...|9137001127547803884|      297.0|      Debit Card|      280|
|   3315976|2024-06-04 01:11:...|8093574796579720949|      995.0|     Credit Card|      497|
|   5749164|2024-06-04 01:11:...|8778815353880623063|      614.0|     Credit Card|      616|
|   7251024|2024-06-04 01:11:...|5982705684848107909|      805.0|      Debit Card|      348|
|   1116733|2024-06-04 01:11:...|3215645724097092988|      729.0|     Credit Card|      913|
|    227716|2024-06-04 01:11:...|8034821542370552879|      365.0|Cash 

In [ ]:
window(sensor_df.CreatedTime, "15 minutes", "5 minute")

window(rQuery.OrderDate, '15 minutes', '5 minutes')

In [30]:
 from pyspark.sql.functions import window, count

display(
    rQuery.withWatermark('OrderDate', '30 minutes').groupby('PaymentMethod',window(rQuery.OrderDate, '15 minutes', '5 minutes'))
    .agg(count('*'))
    .select(f.col("PaymentMethod"), 
            f.col("window.start").alias('startingTime'),
            f.col("window.end").alias("endTime"),
            f.col("count(1)").alias('paymentTypeCount')
    
))

StatementMeta(, , , Cancelled, )

SynapseWidget(Synapse.DataFrame, 0a15c68e-a40f-41d6-873f-ffc76a55c560)